In [1]:
#coding:utf-8

#import library
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from datetime import datetime as dt
import datetime
import configparser
import requests
import subprocess
import os

#variable
config = configparser.ConfigParser()
config.read('../config/kyotei.config')

from_date = config.get('date', 'from_date')
to_date = config.get('date', 'to_date')

download_url = config.get('file', 'download_url_2')
file_dir = config.get('file', 'file_dir')
file_name_race_result = config.get('file', 'file_name_race_result')
file_name_lzh = ''
file_name_txt = ''

db_connect_info = config.get('db', 'db_connect_info')
db_host = config.get('db', 'db_host')
db_port = config.get('db', 'db_port')
db_dbname = config.get('db', 'db_dbname')
db_user = config.get('db', 'db_user')
db_password = config.get('db', 'db_password')

#-----------------------------
#function for download and clean raceinfo file
#-----------------------------
def DownloadFile():
    global file_name_lzh
    global file_name_txt
    
    file_name_lzh = 'b' + race_date[2:8] + '.lzh'
    file_name_txt = 'b' + race_date[2:8] + '.txt'

    #download file
    url = download_url + race_date[0:6] +  '/'+ file_name_lzh
    response = requests.get(url)
    with open(file_dir + file_name_lzh, 'wb') as saveFile:
        saveFile.write(response.content)
    
    #lha file
    cmd = "lha ew=%s %s" % (file_dir, file_dir + file_name_lzh)
    returncode = subprocess.call(cmd, shell=True)
    
    #encoding file
    cmd = "iconv -f SHIFT_JIS -t UTF-8 %s > %s" % (file_dir + file_name_txt, file_dir + file_name_race_result)
    returncode = subprocess.call(cmd, shell=True)

def CleanFile():
    #remove download_file
    cmd = "rm %s" % (file_dir + 'b*')
    returncode = subprocess.call(cmd, shell=True)

    #remove encoding file
    cmd = "rm %s" % (file_dir + file_name_race_result)
    returncode = subprocess.call(cmd, shell=True)

#-----------------------------
#function for set raceinfo
#-----------------------------
def SetRaceResult():
    race_result_attribute = pd.DataFrame(columns=['race_date', 'kaijo', 'race_no', 'entry_order', 'toroku_no', 'age', 'branch', 'weight', 'player_class', 'win_per', 'win_per_within_2', 'win_per_kaijo', 'win_per_kaijo_within_2', 'motor_win_per_within_2', 'boat_win_per_within_2'])
    #debug abc=1
    with open(file_dir + file_name_race_result, 'r') as f:
        for line in f:
            '''
            set race_result variable
                date
                kaijo
                race_no
                entry_order
                toroku_no
                age
                branch
                weight
                player_class
                win_per
                win_per_within_2
                win_per_kaijo
                win_per_kaijo_within_2
                motor_win_per_within_2
                boat_win_per_within_2
            ''' 
            #debug print(str(abc) + ' ' + str(len(line)))
            #debug abc += 1
            if line.find('BGN') > -1:
                kaijo = line[0:3]
            
            if line[2:3]=='Ｒ':
                race_no = int(line[0:2])
            
            if len(line) == 74 and line[0:1].isdigit() == True and 1 <= int(line[0:1]) <= 6:
                entry_order = int(line[0:1])
                toroku_no = int(line[2:6])
                age = int(line[10:12])
                branch = line[12:14]
                weight = int(line[14:16])
                player_class = line[16:18]
                win_per = float(line[18:23])
                win_per_within_2 = float(line[23:29])
                win_per_kaijo = float(line[29:35])
                win_per_kaijo_within_2 = float(line[35:40])
                motor_win_per_within_2 = float(line[44:49])
                boat_win_per_within_2 = float(line[53:58])
                
                #set race_result dataframe 
                race_result_attribute_tmp = pd.DataFrame([[race_date, kaijo, race_no, entry_order, toroku_no, age, branch, weight, player_class, win_per, win_per_within_2, win_per_kaijo, win_per_kaijo_within_2, motor_win_per_within_2, boat_win_per_within_2]], columns=['race_date', 'kaijo', 'race_no', 'entry_order', 'toroku_no', 'age', 'branch', 'weight', 'player_class', 'win_per', 'win_per_within_2', 'win_per_kaijo', 'win_per_kaijo_within_2', 'motor_win_per_within_2', 'boat_win_per_within_2'])
                race_result_attribute = race_result_attribute.append(race_result_attribute_tmp)
                
    return race_result_attribute
 
#-----------------------------
#functino for postgresql
#-----------------------------    
def Pands2Postgre(race_result_attribute):
    engine=create_engine(db_connect_info)

    race_result_attribute.to_sql("wk_race_result_attribute", engine, if_exists="replace", index=False)
    
    
def ExecuteSql(sqltext):
    conn = psycopg2.connect("host=" + db_host + " port=" + db_port + " dbname=" + db_dbname + " user=" + db_user + " password=" + db_password)
    cur = conn.cursor()

    with open(sqltext,'r') as f:
        sql_race_result = f.read()
    
        cur.execute(sql_race_result)
        
        conn.commit()
        cur.close()
        conn.close()
    

#-----------------------------
#function for main
#-----------------------------
def main():
    
    #race result to pandas
    race_result_attribute = SetRaceResult()
    
    #from pandas to postgresql (copy)
    Pands2Postgre(race_result_attribute)
    
    #insert race_result_attribute
    ExecuteSql('../sql/delete_race_result_attribute.sql')
    ExecuteSql('../sql/insert_race_result_attribute.sql')
    
    #clean file
    CleanFile()
    
    return race_result_attribute

    
#-----------------------------
#execute
#-----------------------------
if __name__ == '__main__':
    
    while dt.strptime(from_date, '%Y%m%d').strftime('%Y%m%d') <= dt.strptime(to_date, '%Y%m%d').strftime('%Y%m%d'):
        
        race_date = from_date
        
        DownloadFile()
        print(file_dir + file_name_txt)
        
        if os.path.isfile(file_dir + file_name_txt) == False:
            from_date = (dt.strptime(from_date, '%Y%m%d') + datetime.timedelta(days=1)).strftime('%Y%m%d')
            print(str(race_date) + ' pass')
            continue
        
        main()
        
        from_date = (dt.strptime(from_date, '%Y%m%d') + datetime.timedelta(days=1)).strftime('%Y%m%d')
        
        print(str(race_date) + ' complete')
    
    print('ALL complete')
    

../file/b171003.txt
a
aa


KeyboardInterrupt: 

In [13]:
#coding:utf-8

#import library
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from datetime import datetime as dt
import datetime
import configparser
import requests
import subprocess
import os

#variable
config = configparser.ConfigParser()
config.read('../config/kyotei.config')

from_date = config.get('date', 'from_date')
to_date = config.get('date', 'to_date')

download_url = config.get('file', 'download_url_2')
file_dir = config.get('file', 'file_dir')
file_name_race_result = config.get('file', 'file_name_race_result')
file_name_lzh = ''
file_name_txt = ''

db_connect_info = config.get('db', 'db_connect_info')
db_host = config.get('db', 'db_host')
db_port = config.get('db', 'db_port')
db_dbname = config.get('db', 'db_dbname')
db_user = config.get('db', 'db_user')
db_password = config.get('db', 'db_password')

#-----------------------------
#function for download and clean raceinfo file
#-----------------------------
def DownloadFile():
    global file_name_lzh
    global file_name_txt
    
    file_name_lzh = 'b' + race_date[2:8] + '.lzh'
    file_name_txt = 'b' + race_date[2:8] + '.txt'

    #download file
    url = download_url + race_date[0:6] +  '/'+ file_name_lzh
    response = requests.get(url)
    with open(file_dir + file_name_lzh, 'wb') as saveFile:
        saveFile.write(response.content)
    
    #lha file
    cmd = "lha ew=%s %s" % (file_dir, file_dir + file_name_lzh)
    returncode = subprocess.call(cmd, shell=True)
    
    #encoding file
    cmd = "iconv -f SHIFT_JIS -t UTF-8 %s > %s" % (file_dir + file_name_txt, file_dir + file_name_race_result)
    returncode = subprocess.call(cmd, shell=True)

def CleanFile():
    #remove download_file
    cmd = "rm %s" % (file_dir + 'k*')
    returncode = subprocess.call(cmd, shell=True)

    #remove encoding file
    cmd = "rm %s" % (file_dir + file_name_race_result)
    returncode = subprocess.call(cmd, shell=True)

#-----------------------------
#function for set raceinfo
#-----------------------------
def SetRaceResult():
    race_result_attribute = pd.DataFrame(columns=['race_date', 'kaijo', 'race_no', 'entry_order', 'toroku_no', 'age', 'branch', 'weight', 'player_class', 'win_per', 'win_per_within_2', 'win_per_kaijo', 'win_per_kaijo_within_2', 'motor_win_per_within_2', 'boat_win_per_within_2'])
    abc=1
    with open(file_dir + file_name_race_result, 'r') as f:
        for line in f:
            '''
            set race_result variable
                date
                kaijo
                race_no
                entry_order
                toroku_no
                age
                branch
                weight
                player_class
                win_per
                win_per_within_2
                win_per_kaijo
                win_per_kaijo_within_2
                motor_win_per_within_2
                boat_win_per_within_2
            ''' 
            print(str(abc) + ' ' + str(len(line)) + ' ' + line[2:3])
            abc += 1
            if line.find('BGN') > -1:
                kaijo = line[0:3]
            
            if line[2:3]=='Ｒ':
                race_no = int(line[0:2])
                print('race')
            
            if len(line) == 74 and line[0:1].isdigit() == True and 1 <= int(line[0:1]) <= 6:
                entry_order = int(line[0:1])
                toroku_no = int(line[2:6])
                age = int(line[10:12])
                branch = line[12:14]
                weight = int(line[14:16])
                player_class = line[16:18]
                win_per = float(line[18:23])
                win_per_within_2 = float(line[23:29])
                win_per_kaijo = float(line[29:35])
                win_per_kaijo_within_2 = float(line[35:40])
                motor_win_per_within_2 = float(line[44:49])
                boat_win_per_within_2 = float(line[53:58])
                
                #set race_result dataframe 
                race_result_attribute_tmp = pd.DataFrame([[race_date, kaijo, race_no, entry_order, toroku_no, age, branch, weight, player_class, win_per, win_per_within_2, win_per_kaijo, win_per_kaijo_within_2, motor_win_per_within_2, boat_win_per_within_2]], columns=['race_date', 'kaijo', 'race_no', 'entry_order', 'toroku_no', 'age', 'branch', 'weight', 'player_class', 'win_per', 'win_per_within_2', 'win_per_kaijo', 'win_per_kaijo_within_2', 'motor_win_per_within_2', 'boat_win_per_within_2'])
                race_result_attribute = race_result_attribute.append(race_result_attribute_tmp)
                
    return race_result_attribute
 
#-----------------------------
#functino for postgresql
#-----------------------------    
def Pands2Postgre(race_result_attribute):
    engine=create_engine(db_connect_info)

    race_result_attribute.to_sql("wk_race_result_attribute", engine, if_exists="replace", index=False)
    
def ExecuteSql(sqltext):
    conn = psycopg2.connect("host=" + db_host + " port=" + db_port + " dbname=" + db_dbname + " user=" + db_user + " password=" + db_password)
    cur = conn.cursor()

    with open(sqltext,'r') as f:
        sql_race_result = f.read()
    
        cur.execute(sql_race_result)
        
        conn.commit()
        cur.close()
        conn.close()
    

#-----------------------------
#function for main
#-----------------------------
def main():
    
    #race result to pandas
    race_result_attribute = SetRaceResult()
    print('a')
    
    #from pandas to postgresql (copy)
    Pands2Postgre(race_result_attribute)
    print('b')
    
    #insert race_result_attribute
    ExecuteSql('../sql/delete_race_result_attribute.sql')
    ExecuteSql('../sql/insert_race_result_attribute.sql')
    print('c')
    
    #clean file
    CleanFile()
    
    return race_result_attribute

    
#-----------------------------
#execute
#-----------------------------
race_result_attribute = SetRaceResult()
    

1 7 A
2 7 B
3 37 ト
4 1 
5 40  
6 1 
7 36  
8 1 
9 56  
10 1 
11 41  
12 1 
13 1 
14 44 Ｒ
race
15 80 -
16 55 選
17 57 登
18 80 -
19 74 4
20 74 3
21 74 3
22 74 4
23 74 4
24 74 3
25 1 
26 44 Ｒ
race
27 80 -
28 55 選
29 57 登
30 80 -
31 74 4
32 74 4
33 74 4
34 74 3
35 74 4
36 74 4
37 1 
38 44 Ｒ
race
39 80 -
40 55 選
41 57 登
42 80 -
43 74 4
44 74 4
45 74 4
46 74 4
47 74 4
48 74 3
49 1 
50 44 Ｒ
race
51 80 -
52 55 選
53 57 登
54 80 -
55 74 4
56 74 3
57 74 4
58 74 4
59 74 3
60 74 4
61 1 
62 44 Ｒ
race
63 80 -
64 55 選
65 57 登
66 80 -
67 74 4
68 74 4
69 74 4
70 74 4
71 74 3
72 74 3
73 1 
74 44 Ｒ
race
75 80 -
76 55 選
77 57 登
78 80 -
79 74 4
80 74 3
81 74 4
82 74 4
83 74 4
84 74 4
85 1 
86 44 Ｒ
race
87 80 -
88 55 選
89 57 登
90 80 -
91 74 4
92 74 3
93 74 4
94 74 3
95 74 4
96 74 4
97 1 
98 44 Ｒ
race
99 80 -
100 55 選
101 57 登
102 80 -
103 74 4
104 74 4
105 74 3
106 74 3
107 74 4
108 74 3
109 1 
110 44 Ｒ
race
111 80 -
112 55 選
113 57 登
114 80 -
115 74 4
116 74 4
117 74 4
118 74 4
119 74 4
120 74 4
121 1 
122 44

In [9]:
line='　１Ｒ  予　　選　　          Ｈ１８００ｍ  電話投票締切予定１０：５２ '
#line='１０Ｒ  予　　選　　          Ｈ１８００ｍ  電話投票締切予定１５：２９ '
#艇 選手 選手  年 支 体級    全国      当地     モーター   ボート   今節成績  早
#番 登番  名   齢 部 重別 勝率  2率  勝率  2率  NO  2率  NO  2率  １２３４５６見
#line='1 4238毒島　誠34群馬52A1 7.71 55.36 8.32 67.86 23 37.87 51 30.77 5 2145        '
#line='2 4762藤原啓史28岡山51B1 5.37 35.21 8.00 66.67 46 26.44 46 38.54 3           11'
'''
                entry_order = int(line[0:1])
                toroku_no = int(line[2:6])
                age = int(line[10:12])
                branch = line[12:14]
                weight = int(line[14:16])
                player_class = line[16:18]
                win_per = float(line[18:23])
                win_per_within_2 = float(line[23:29])
                win_per_kaijo = float(line[29:35])
                win_per_kaijo_within_2 = float(line[35:40])
                motor_win_per_within_2 = float(line[44:49])
                boat_win_per_within_2 = float(line[53:58])
'''
print(line[2:3])
if len(line) == 73 and line[0:1].isdigit() == True and 1 <= int(line[0:1]) <= 6:
    print('ok')

line='1aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'
if line[0:1].isdigit() == True and 1 <= int(line[0:1]) <= 6:
    #if 1 <= int(line[0:1]) <= 6:
    print('ok')


Ｒ
ok
